<a href="https://colab.research.google.com/github/Devininthelab/Toy-Repo/blob/main/LoRA3_loralib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import packages

In [1]:
!pip3 install loralib

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Dataloader

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

BATCH_SIZE = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)


100%|██████████| 170498071/170498071 [00:19<00:00, 8625733.41it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# 3. Modeling

In [4]:
def trainable_params(model):
    total_params = 0
    for param in model.parameters():
        if param.requires_grad:
            total_params += param.numel()

    print(f"Trainable params: {total_params/1e6:.2f} M")

## 3.1 Load pretrained weight Imagenet1k

In [5]:
from torchvision.models import vgg16, VGG16_Weights
vgg16_model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

vgg16_classifier_ckpt = vgg16_model.classifier.state_dict()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 100MB/s] 


In [6]:
trainable_params(vgg16_model)

Trainable params: 138.36 M


In [7]:
trainable_params(vgg16_model.classifier)

Trainable params: 123.64 M


## 3.2 Define LORA

In [8]:
import loralib

lora_classifier =  nn.Sequential(
    loralib.Linear(in_features=25088, out_features=4096, bias=True, r=16),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    loralib.Linear(in_features=4096, out_features=4096, bias=True, r=16),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    loralib.Linear(in_features=4096, out_features=1000, bias=True, r=16)
)

## 3.3: Load pre_trained weights to LoRA clf

In [9]:
lora_classifier.load_state_dict(vgg16_classifier_ckpt, strict=False)


_IncompatibleKeys(missing_keys=['0.lora_A', '0.lora_B', '3.lora_A', '3.lora_B', '6.lora_A', '6.lora_B'], unexpected_keys=[])

## 3.4: Add output for cifar10

In [10]:
new_classififer = nn.Sequential(
    *lora_classifier,
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    loralib.Linear(in_features=1000, out_features=10, bias=True)
)

In [11]:
trainable_params(new_classififer)

Trainable params: 0.70 M


## 3.5: Wrap with fe

In [12]:
class CLS_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.features = vgg16_model.features.eval()
        for param in self.features.parameters():
            param.requires_grad_(False)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = new_classififer

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [13]:
model = CLS_model().to(device)

In [14]:
trainable_params(model)

Trainable params: 0.70 M


# 4. Training

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [16]:
model.train()

start = time.time()
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{10}], Average Loss: {running_loss / len(trainloader):.4f}, GPU used: {torch.cuda.memory_allocated(0)/1e9:.2f} G')

print('Finished Training')
print(f'Training time: {time.time() - start:.2f} s')

Epoch [1/10], Average Loss: 1.5450, GPU used: 0.59 G
Epoch [2/10], Average Loss: 1.2925, GPU used: 0.59 G
Epoch [3/10], Average Loss: 1.2137, GPU used: 0.59 G
Epoch [4/10], Average Loss: 1.1491, GPU used: 0.59 G
Epoch [5/10], Average Loss: 1.1118, GPU used: 0.59 G
Epoch [6/10], Average Loss: 1.0814, GPU used: 0.59 G
Epoch [7/10], Average Loss: 1.0606, GPU used: 0.59 G
Epoch [8/10], Average Loss: 1.0339, GPU used: 0.59 G
Epoch [9/10], Average Loss: 1.0120, GPU used: 0.59 G
Epoch [10/10], Average Loss: 1.0029, GPU used: 0.59 G
Finished Training
Training time: 215.79 s


# 5. Eval

In [17]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Accuracy of the model on the 10000 test images: 65.1 %
